In [ ]:
import sys
sys.path.append("lib/")
import random
import time
import csv
import math
import imp
import Sorts
from Sorts import dual_pivot
from Sorts import introsort
from sort_inspector import Sequence

This function generates tuples with lists to sort, and info about each as tuples:

In [ ]:
def trial_gen(sizes,types=['shuffle']):
    for n in sizes:
        for typ in types:
            if typ == 'sorted':
                yield (typ,n,list(range(0,n)))
            elif typ == 'reverse':
                yield (typ,n,list(range(n-1,-1,-1)))
            elif typ == 'shuffle':
                t = list(range(0,n))
                random.shuffle(t)
                yield (typ,n,t)
            else: #typ is integer number of options
                t = [ random.randrange(0,typ) for x in range(0,n)]
                yield ('pick',len(set(t)),t)

Generates logarithmically-spaced sequences:

In [ ]:
def log_range(start,end,jump = math.sqrt(math.e)):
    x = float(start)
    while x < end:
        yield int(x)
        x *= jump

Given a sort function, a sequence of tuples as created by `trial_gen()`, and a set of "modes" (which consist of additional output columns, and additional arguments to the sort function, generates a sequence of output rows including timings and swap/compare counts.

In [ ]:
def run_tests(sortfn,trials,modes=[([],[],{})]):
    for (typ,n_distinct,vals) in trials:
        for (mode_info,mode_args,mode_kwargs) in modes:
            randstate = random.getstate()
            T0 = time.process_time()
            sortfn(vals[:],*mode_args,**mode_kwargs)
            T1 = time.process_time()
            elapsed = T1 - T0
            mons = Sequence(vals)
            random.setstate(randstate)
            sortfn(mons,*mode_args,**mode_kwargs)
            yield [len(vals),typ,n_distinct,elapsed,mons.count_swaps(),mons.count_comparisons()] + mode_info

In [ ]:
list(run_tests(dual_pivot,trial_gen(range(1,20,9),['sorted','shuffle',2])))

In [ ]:
def write_results(out,results,addl_headers=[]):
    with open(out,'w',newline ='') as f:
        writer = csv.writer(f, quoting=csv.QUOTE_MINIMAL)
        writer.writerow(['len','type','n_distinct','time','swaps','comparisons']+addl_headers)
        for row in results:
            writer.writerow(row)

In [ ]:
write_results('./dual_pivot.csv',
              results = run_tests(dual_pivot, trial_gen(log_range(10,50000),
                                                        ['sorted','reverse','shuffle',2,5,10,100]
                                                       ))
            )

In [ ]:
write_results('./introsort.csv',
               results = run_tests(introsort, trial_gen(log_range(10,50000),
                                                        ['sorted','reverse','shuffle',2,5,10,100]
                                                       ))
             )